In [79]:
import pandas as pd
import os

#### DEFINE
output_name = "0MATERIAL_ATTR"


In [80]:
with open("B2D.txt", "r", encoding="utf-8") as file:
    ecc_text = file.read()
with open("B4P.txt", "r", encoding="utf-8") as file:
    hana_text = file.read()

print(os.listdir())

['B2D.txt', 'B4P.txt', 'fucking_output.xlsx', 'SAPTableCompare.ipynb', 't1.txt', 'table1.txt']


In [74]:
def export_df(text):
        ls = [i.strip() for i in text.split("\n") 
                if 
                        ("Append structure " not in i) 
                        and ("Include structure ") not in i 
                        and (len(i) > 1)
                ]
        ls_ls = [i.split(" ") for i in ls]

        #####Define List
        set1 = set([i[-2] for i in ls_ls]) # set2 =  set([i.split(" ")[-3] for i in ls])
        ls1 = [i for i in list(set1) if any(j.isalpha() for j in i)]
        ls1.extend(["QUAN", "DEC"])
        type_ls = list(set(ls1)) 

        ls_name = [ i[0] for i in ls_ls]

        ls_type = []
        for i in ls_ls:
                if i[-2] in type_ls:
                        ls_type.append(f'{i[-2]} {i[-1]}')
                else:
                        ls_type.append(f'{i[-3]} {i[-2]},{i[-1]}')


        ls_combine = []
        for i in range(0,len(ls_ls)):
                ls_combine.append(f'{i} ' + ls_name[i]+" "+ls_type[i])
        ls_combine_ls = [i.split(" ") for i in ls_combine]
        
        return ls_combine_ls

In [75]:
#### Read files and apply def
hana = export_df(hana_text)
ecc = export_df(ecc_text)

# Prepare Dataframe
df_ecc = pd.DataFrame(data=ecc,columns=["e_index","e_name","e_type","e_lgt"])
df_hana = pd.DataFrame(data=hana,columns=["h_index","h_name","h_type","h_lgt"])


In [76]:
#### Check Field Mismatch
ecc_name = set([i[1] for i in ecc])
hana_name = set([i[1] for i in hana])

ecc_u = []
hana_u = []
if ecc_name == hana_name:
    print("Both DataFrames have the same columns.")
else:
    for i in ecc_name - hana_name:
        ecc_u.append(i)
    for i in hana_name - ecc_name:
        hana_u.append(i)

# Make same lenght
max_len = max(len(ecc_u), len(hana_u))
ecc_u += [""] * (max_len - len(ecc_u))  # Pad with empty string
hana_u += [""] * (max_len - len(hana_u))

# Prepare Dataframe
df_mismatched_fields = pd.DataFrame({"Only in HANA": hana_u,"Only in ECC": ecc_u})

# print(df_mismatched_fields.to_string(index=False))  # Properly formatted DataFrame output

In [77]:
#### Check Type Mismatch
ecc_dict = {row[1]: row for row in ecc}  # {C1: [...], C3: [...]}
hana_dict = {row[1]: row for row in hana}  # {C1: [...], C3: [...]}
matching_keys = set(ecc_dict.keys()) & set(hana_dict.keys())

data = []
for i in matching_keys:
    e_row = ecc_dict[i]
    h_row = hana_dict[i]
    if (e_row[3] != h_row[3]) or (e_row[2] != h_row[2]):
        data.append({
            "e_name": e_row[1], "e_type": e_row[2], "e_lgt": e_row[3],
            "trans": ">>>",
            "h_name": h_row[1], "h_type": h_row[2], "h_lgt": h_row[3]
        })

# Convert to DataFrame
df_mismatched_types = pd.DataFrame(data)

# print(df.to_string(index=False))


In [81]:
# Define Excel file path
excel_path = f"{output_name}.xlsx"

# Write to Excel with multiple sheets
with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
    df_ecc.to_excel(writer, sheet_name="ecc", index=False)
    df_hana.to_excel(writer, sheet_name="hana", index=False)
    df_mismatched_types.to_excel(writer, sheet_name="mismatched_types", index=False)
    df_mismatched_fields.to_excel(writer, sheet_name="mismatched_fields", index=False)


print(f"Excel file '{excel_path}'.")


Excel file '0MATERIAL_ATTR.xlsx'.


In [ ]:
# ecc_ls = [i.strip() for i in ecc_text.split("\n") 
#             if 
#                 ("Append structure " not in i) 
#                 and ("Include structure ") not in i 
#                 and (len(i) > 1)
#         ]
# ecc_ls_ls = [i.split(" ") for i in ecc_ls]

# #####Define List
# set1 = set([i[-2] for i in ecc_ls_ls]) # set2 =  set([i.split(" ")[-3] for i in ecc_ls])
# ls1 = [i for i in list(set1) if i.isalpha()]
# ls1.extend(["QUAN", "DEC"])
# type_ls = list(set(ls1)) 
# # print(type_ls)

# ecc_ls_name = [ i[0] for i in ecc_ls_ls]

# ecc_ls_type = []
# for i in ecc_ls_ls:
#         if i[-2] in type_ls:
#                 ecc_ls_type.append(f'{i[-2]} {i[-1]}')
#         else:
#                 ecc_ls_type.append(f'{i[-3]} {i[-2]} {i[-1]}')


# ecc_ls_combine = []
# for i in range(0,len(ecc_ls_ls)):
#         ecc_ls_combine.append(f'{i} ' + ecc_ls_name[i]+","+ecc_ls_type[i])

#         ecc_ls = [i.strip() for i in hana_text.split("\n") 
#         if 
#             ("Append structure " not in i) 
#             and ("Include structure ") not in i 
#             and (len(i) > 1)
#     ]
# ecc_ls_ls = [i.split(" ") for i in ecc_ls]

# #####Define List
# set1 = set([i[-2] for i in ecc_ls_ls]) # set2 =  set([i.split(" ")[-3] for i in ecc_ls])
# ls1 = [i for i in list(set1) if i.isalpha()]
# ls1.extend(["QUAN", "DEC"])
# type_ls = list(set(ls1)) 
# # print(type_ls)

# ecc_ls_name = [ i[0] for i in ecc_ls_ls]

# ecc_ls_type = []
# for i in ecc_ls_ls:
#     if i[-2] in type_ls:
#             ecc_ls_type.append(f'{i[-2]} {i[-1]}')
#     else:
#             ecc_ls_type.append(f'{i[-3]} {i[-2]} {i[-1]}')


# ecc_ls_combine = []
# for i in range(0,len(ecc_ls_ls)):
#     ecc_ls_combine.append(f'{i} ' + ecc_ls_name[i]+","+ecc_ls_type[i])